In [1]:
from arcpy import *
from arcpy import env
from arcpy.sa import *
import pandas as pd
import numpy as np
import os, datetime
from osgeo import ogr
import rasterio
import xarray as xr
from rasterio.transform import from_origin
import subprocess, time
from tqdm import tqdm
import shutil

In [ ]:
substances = ["BC", "BENZENE", "CO", "NH3", "NMVOCS", "NOx", "OC", "SO2", "VOCS"]
for substance in substances:
    os.makedirs(f"Input_2/{substance}", exist_ok=True)
path_excel_files = os.listdir(r"G:/NASA_TUTORIAL_SPACE/EMISSION3/ANT/EXCEL/CB/")
for path_excel_file in path_excel_files:
    parts = path_excel_file.split('_'); substance = parts[-1][:-5]
    src = r"G:/NASA_TUTORIAL_SPACE/EMISSION3/ANT/EXCEL/CB/" + path_excel_file
    if (substance == "benzene"):
        shutil.copy(src, "Input_2/BENZENE")
    elif (substance == "co"):
        shutil.copy(src, "Input_2/CO")
    elif (substance == "nox"):
        shutil.copy(src, "Input_2/NOx")
    elif (substance == "bc"):
        shutil.copy(src, "Input_2/BC")
    elif (substance == "oc"):
        shutil.copy(src, "Input_2/OC")
    elif (substance == "so2"):
        shutil.copy(src, "Input_2/SO2")
    elif (substance == "nmvocs"):
        shutil.copy(src, "Input_2/NMVOCS")
    elif (substance == "vocs"):
        shutil.copy(src, "Input_2/VOCS")
    elif (substance == "nh3"):
        shutil.copy(src, "Input_2/NH3")

In [ ]:
for substance in substances:
    data_dict = {}; lon = []; lat = []
    path_excel = fr"Input_2/{substance}/" #SUA CHAT
    excel_files = os.listdir(path_excel)
    df = pd.read_excel(path_excel + excel_files[0], header=None)
    df.columns = ["Year", "Month", "Day", "Hour", "Lat", "Lon", "Substance", "Value"]
    df = pd.DataFrame(df.groupby(['Lat', 'Lon'])["Value"].sum().reset_index(), columns=['Lat', 'Lon', 'Value'])
    for index in range(df['Lat'].size):
        data_dict[(df['Lat'][index], df['Lon'][index])] = 0
        lon.append(df['Lon'][index]); lat.append(df['Lat'][index])

    
    day_month = 31
    day_base = [0, 1, 2, 3, 4, 5, 6, 7]
    for excel_file in excel_files:
        print(excel_file)
        path_excel_file = path_excel + excel_file; day = 0
        df_sum = pd.read_excel(path_excel_file, header=None, sheet_name = f"Day_{day+1}")
        df_sum.columns = ["Year", "Month", "Day", "Hour", "Lat", "Lon", "Substance", "Value"]
        df_sum = pd.DataFrame(df.groupby(['Lat', 'Lon'])["Value"].sum().reset_index(), columns=['Lat', 'Lon', 'Value'])
        df_sum["Value"] = 0
        for day in range(7):
            day_actual = day+1; multiplier = int((day_month-day_base[day_actual])/7) + 1
            df = pd.read_excel(path_excel_file, header=None, sheet_name = f"Day_{day_actual}")
            df.columns = ["Year", "Month", "Day", "Hour", "Lat", "Lon", "Substance", "Value"]
            df = pd.DataFrame(df.groupby(['Lat', 'Lon'])["Value"].sum().reset_index(), columns=['Lat', 'Lon', 'Value'])
            df_sum['Value'] += df['Value']*float(multiplier)
        for _, row in df_sum.iterrows():
            data_dict[(row['Lat'], row['Lon'])] = data_dict[(row['Lat'], row['Lon'])] + row['Value']




    # Open the shapefile
    driver = ogr.GetDriverByName('ESRI Shapefile')
    dataSource = driver.Open(r"G:\NASA_TUTORIAL_SPACE\TEMP\NASA\0_Result\PM25\Shapefile\OREGON_GRID\polygon.shp", 1)  # 1 is read/write
    ### Đường dẫn shapefile cần phải sửa

    # Get the layer
    layer = dataSource.GetLayer()

    # Check if substance field exists and delete it if it does
    layer_defn = layer.GetLayerDefn()
    field_names = [layer_defn.GetFieldDefn(i).GetName() for i in range(layer_defn.GetFieldCount())]

    if substance in field_names:
        layer.DeleteField(layer_defn.GetFieldIndex(substance))

    # Define the new field              
    fldDef = ogr.FieldDefn(substance, ogr.OFTReal)
    layer.CreateField(fldDef)
    # Update the new field with values from the DataFrameimport numpy as np
    for i, feature in enumerate(layer):
        feature.SetField(substance, data_dict[(lat[i], lon[i])]*2678.4)
        layer.SetFeature(feature)  # Save the changes to the feature
    # Save and close the data source
    dataSource = None

Ant_PEC_awb_10_bc.xlsx
Ant_PEC_ene_10_bc.xlsx
Ant_PEC_fef_10_bc.xlsx
Ant_PEC_ind_10_bc.xlsx
Ant_PEC_res_10_bc.xlsx
Ant_PEC_shp_10_bc.xlsx
Ant_PEC_swd_10_bc.xlsx
Ant_PEC_tro_10_bc.xlsx
Ant_BENZ_awb_10_benzene.xlsx
Ant_BENZ_ene_10_benzene.xlsx
Ant_BENZ_fef_10_benzene.xlsx
Ant_BENZ_ind_10_benzene.xlsx
Ant_BENZ_res_10_benzene.xlsx
Ant_BENZ_shp_10_benzene.xlsx
Ant_BENZ_slv_10_benzene.xlsx
Ant_BENZ_swd_10_benzene.xlsx
Ant_BENZ_tro_10_benzene.xlsx
Ant_CO_awb_10_co.xlsx
Ant_CO_ene_10_co.xlsx
Ant_CO_fef_10_co.xlsx
Ant_CO_ind_10_co.xlsx
Ant_CO_res_10_co.xlsx
Ant_CO_shp_10_co.xlsx
Ant_CO_swd_10_co.xlsx
Ant_CO_tro_10_co.xlsx
Ant_NH3_agl_10_nh3.xlsx
Ant_NH3_ags_10_nh3.xlsx
Ant_NH3_awb_10_nh3.xlsx
Ant_NH3_ene_10_nh3.xlsx
Ant_NH3_ind_10_nh3.xlsx
Ant_NH3_res_10_nh3.xlsx
Ant_NH3_shp_10_nh3.xlsx
Ant_NH3_slv_10_nh3.xlsx
Ant_NH3_swd_10_nh3.xlsx
Ant_NH3_tro_10_nh3.xlsx
Ant_VOC_INV_agl_10_nmvocs.xlsx
Ant_VOC_INV_ags_10_nmvocs.xlsx
Ant_VOC_INV_awb_10_nmvocs.xlsx
Ant_VOC_INV_ene_10_nmvocs.xlsx
Ant_VOC_INV_fef

In [10]:
data_dict = {}; lon = []; lat = []
path_excel = r"Input/VOCS/" #SUA CHAT
excel_files = os.listdir(path_excel)
df = pd.read_excel(path_excel + excel_files[0], header=None)
df.columns = ["Year", "Month", "Day", "Hour", "Lat", "Lon", "Substance", "Value"]
df = pd.DataFrame(df.groupby(['Lat', 'Lon'])["Value"].sum().reset_index(), columns=['Lat', 'Lon', 'Value'])
for index in range(df['Lat'].size):
    data_dict[(df['Lat'][index], df['Lon'][index])] = 0
    lon.append(df['Lon'][index]); lat.append(df['Lat'][index])

In [11]:
day_month = 31
day_base = [0, 1, 2, 3, 4, 5, 6, 7]
for excel_file in excel_files:
    print(excel_file)
    path_excel_file = path_excel + excel_file; day = 0
    df_sum = pd.read_excel(path_excel_file, header=None, sheet_name = f"Day_{day+1}")
    df_sum.columns = ["Year", "Month", "Day", "Hour", "Lat", "Lon", "Substance", "Value"]
    df_sum = pd.DataFrame(df.groupby(['Lat', 'Lon'])["Value"].sum().reset_index(), columns=['Lat', 'Lon', 'Value'])
    df_sum["Value"] = 0
    for day in range(7):
        day_actual = day+1; multiplier = int((day_month-day_base[day_actual])/7) + 1
        df = pd.read_excel(path_excel_file, header=None, sheet_name = f"Day_{day_actual}")
        df.columns = ["Year", "Month", "Day", "Hour", "Lat", "Lon", "Substance", "Value"]
        df = pd.DataFrame(df.groupby(['Lat', 'Lon'])["Value"].sum().reset_index(), columns=['Lat', 'Lon', 'Value'])
        df_sum['Value'] += df['Value']*float(multiplier)
    for _, row in df_sum.iterrows():
        data_dict[(row['Lat'], row['Lon'])] = data_dict[(row['Lat'], row['Lon'])] + row['Value']

Ant_VOC_INV_awb_7_other_vocs.xlsx
Ant_VOC_INV_ene_7_other_vocs.xlsx
Ant_VOC_INV_fef_7_other_vocs.xlsx
Ant_VOC_INV_ind_7_other_vocs.xlsx
Ant_VOC_INV_res_7_other_vocs.xlsx
Ant_VOC_INV_shp_7_other_vocs.xlsx
Ant_VOC_INV_slv_7_other_vocs.xlsx
Ant_VOC_INV_swd_7_other_vocs.xlsx
Ant_VOC_INV_tro_7_other_vocs.xlsx


In [12]:
data_dict

{(41.8587564095, -122.591813718): 0.12615818336219184,
 (41.864374276, -119.048572769): 0.02857804349126607,
 (41.873909992, -120.820265382): 0.00043265691672243555,
 (41.8792133005, -124.024178305): 0.012549744125538428,
 (41.9104429574, -118.706575102): 0.015200677492819713,
 (41.9143226694, -122.252376132): 0.0011559251068377983,
 (41.9247346049, -120.47942745): 0.0011335134071916281,
 (41.938594866, -123.685802609): 0.0006994196226231274,
 (41.955587115, -118.364133409): 0.00028018678692576053,
 (41.9689765237, -121.912403881): 0.012746146387689394,
 (41.9746424385, -120.138102617): 0.00025636828067696337,
 (41.9970713388, -123.346859102): 0.00026553246083687633,
 (41.9998070469, -118.021257324): 3.865933126121009e-05,
 (42.022717346, -121.571909566): 0.044780063666281913,
 (42.0236300533, -119.796296171): 0.0038820556448146333,
 (42.0430984623, -117.677955012): 0.0013594885213476548,
 (42.054639451, -123.007355501): 0.0016275242813783507,
 (42.0703856034, -124.443772689): 6.550492

In [13]:
len(data_dict)

354

In [14]:
# Open the shapefile
driver = ogr.GetDriverByName('ESRI Shapefile')
dataSource = driver.Open(r"D:/Code_Result/Result/NASA/0_Result/PM25/Shapefile/OREGON_GRID/polygon.shp", 1)  # 1 is read/write
### Đường dẫn shapefile cần phải sửa

# Get the layer
layer = dataSource.GetLayer()

# Check if 'VOCS' field exists and delete it if it does
layer_defn = layer.GetLayerDefn()
field_names = [layer_defn.GetFieldDefn(i).GetName() for i in range(layer_defn.GetFieldCount())]

if 'VOCS' in field_names:
    layer.DeleteField(layer_defn.GetFieldIndex('VOCS'))

# Define the new field
fldDef = ogr.FieldDefn('VOCS', ogr.OFTReal)
layer.CreateField(fldDef)
# Update the new field with values from the DataFrameimport numpy as np
for i, feature in enumerate(layer):
    feature.SetField('VOCS', data_dict[(lat[i], lon[i])]*2678.4)
    layer.SetFeature(feature)  # Save the changes to the feature
# Save and close the data source
dataSource = None

AttributeError: 'NoneType' object has no attribute 'GetLayer'